In [ ]:
#!pip install EnsemblePursuit
from EnsemblePursuit import EnsemblePursuit

# Introduction

In this notebook we work with a large-scale calcium imaging recording from the primary visual cortex V1 of the mouse available publicly https://figshare.com/articles/Recording_of_19_000_neurons_across_mouse_visual_cortex_during_sparse_noise_stimuli/9505250 

We apply a sparse matrix factorization algorithm called EnsemblePursuit to extract ensembles of correlated cells from this dataset. The algorithm is available here https://github.com/MouseLand/EnsemblePursuit It can be pip installed ('pip install EnsemblePursuit').

The following notebook explored the relationship between the spatial organization of cells in the ensemble and their relationship to stimuli in receptive fields and behavioral data in the form of motion energy of the mouse's face extracted via the SVD algorithm, https://github.com/MouseLand/EnsemblePursuit/blob/master/Notebooks/BehaviorStimulusCorrsWithSpatialSpreadOfEnsemble.ipynb Specifically, we use ridge regression to map from stimulus and behavioral components to the average activity of the cells in ensemble and quantify how well the learned models can predict the activity of the ensemble on test data (quantified through correlation of the predicted timeseries with the actual timeseries). 

We found that ensembles that exhibit linear receptive fields are localized in space while ensembles that are best explained by behavior display higher spatial variance. 

In this notebook we take inspiration from a paper that found that the synapses between neurons with similar receptive fields are very strong, while synapses that are formed between cells that do not show correlation in their responses are weak. 

We create an artificial neuron that receives strong inputs from a "sensory" ensemble (ensemble with a receptive field) and weak inputs from a spatially spread out "behavioral ensemble" (ensemble taht is well predicted by behavioral motion SVD's). We endow the neuron with the BCM plasticity rule and study the evolution of synaptic weights and the change in recepti

# Extracting Ensembles of Correlated Cells